In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install biopython
!pip install pandas
!pip install collection
!pip install numpy

In [ ]:
import random
import numpy as np
import pandas as pd

In [71]:
numOrganismos = 0
arqSaida = ""
arq = open("/content/drive/MyDrive/pr2_version_5.0.0_SSU_dada2.fasta", "r")
linhas = arq.readlines()
#random.seed(22042001)
organismos = {}
organismos_familia = {}

for i in range(0, len(linhas), 2):
  organismos_familia[linhas[i].split(";")[7]] = linhas[i].split(";")[6]
  organismos[linhas[i].split(";")[6] + "  " + linhas[i].split(";")[7]] = linhas[i+1]



In [72]:
# k_mers - para o database
def getKmers(organismos, tamanho):
  k_mers = {}

  for genero, sequencia in organismos.items():
    lista_k_mers = {}
    for i in range (0, len(sequencia)-tamanho+1):
      lista_k_mers[sequencia[i:i+tamanho]] = lista_k_mers.get(sequencia[i:i+tamanho],0) + 1
    k_mers[genero] = lista_k_mers

  return k_mers

# k_mers - para a sequência a ser classificada
def getKmersSeq(sequencia, tamanho):
  k_mers = {}
  for i in range(0, len(sequencia)-tamanho+1):
    k_mers[sequencia[i:i+tamanho]] = k_mers.get(sequencia[i:i+tamanho],0) + 1
  return k_mers


k_db = getKmers(organismos, 8)
k_cl = getKmersSeq("CGGAGAGGGAGCCTGAGAAACGGCTACCACATCTACGGAAGGCAGCAGGCGCGCAAATTGCCCAATCCTGACACGGGGAGGCAGT", 8)

print(k_db)
print(k_cl)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# k_dbKEY -> Unruhdinium
# k_dbVAL -> {'ATGCTTGT': 1, 'TGCTTGTC': 1, 'GCTTGTCT': 1, 'CTTGTCTC': 1,...}
# k_cl -> {'CGGAGAGG': 1, 'GGAGAGGG': 1, 'GAGAGGGM': 1, 'AGAGGGMM': 1, 'GAGGGMMM': 1, 'AGGGMMMM': 1, 'GGGMMMMM': 1, 'GGMMMMMM': 1, 'GMMMMMMM': 1}


aparece = {}
for k_dbKEY, k_dbVAL in k_db.items():
  intersec = k_dbVAL.keys() & k_cl.keys()
  #print(intersec)
  #print(k_cl)
  aparece_KMER = {}
  for k_clKEY in k_cl.keys():
    aparece_KMER[k_clKEY] = 1 if k_clKEY in intersec else 0
  aparece[k_dbKEY] = aparece_KMER

#print(aparece)


df = (pd.DataFrame(aparece)).transpose().sort_index(axis=1)

print(df)

df.to_csv('/content/drive/MyDrive/tabela.csv', index=True)


#         [K-mer 1] [K-mer2] [K-mer3]
# [Arroz]    1         0       1
# [Alga]
# [Cad]

# dicionario de listas -> genero : [1,2,3,4]

# cada linha com um gênero distinto, tendo o número de linhas iguais ao gêneros distintos totais
# cada coluna vai ser um k-mer da sequência a ser classsificada
# marco com 1 se aquele k-mer existe no gênero, marco 0 se não existir
# pintar os que forem 1 de roxo, e os que forem 0 de branco (?)